# Semantic Search using ELSER text expansion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/notebooks/search/03-ELSER.ipynb)


Learn how to use the [ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) for text expansion-powered semantic search.

# 🧰 Requirements

For this example, you will need:

- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?elektra=en-ess-sign-up-page))


# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Setup ELSER
To use ELSER, you must have the [appropriate subscription]() level
for semantic search or the trial period activated.

Follow these [instructions](https://www.elastic.co/guide/en/machine-learning/8.8/ml-nlp-elser.html#trained-model) to download and deploy ELSER in the Kibana UI or using the Dev Tools **Console**.

(Console commands in comments 👇)
<!-- # download elser model

```json
PUT _ml/trained_models/.elser_model_1
{
  "input": {
	"field_names": ["text_field"]
  }
}
``` -->
<!-- # deploy model
```json
POST _ml/trained_models/.elser_model_1/deployment/_start?deployment_id=for_search -->

# Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`


In [ ]:
!pip install elasticsearch

Next, we need to import the modules we need.
🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpass
import json

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [ ]:
print(client.info())

Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Indexing Documents with ELSER

In order to use ELSER on our Elastic Cloud deployment we'll need to create an ingest pipeline that contains an inference processor that runs the ELSER model.
Let's add that pipeline using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method.

In [ ]:

client.ingest.put_pipeline(
    id="elser-ingest-pipeline", 
    description="Ingest pipeline for ELSER",
    processors=[
    {
      "inference": {
        "model_id": ".elser_model_1",
        "target_field": "ml",
        "field_map": {
          "plot": "text_field"
        },
        "inference_config": {
          "text_expansion": {
            "results_field": "tokens"
          }
        }
      }
    }
  ]
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `.elser_model_1`.
- `target_field`: Defines the field where the inference result will be stored. Here, it is set to `ml`.
- `text_expansion`: Configures text expansion options for the inference process.
In this example, the inference results will be stored in a field named "tokens".

## Create index with mappings

To use the ELSER model at index time, we'll need to create an index mapping that supports a [`text_expansion`](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-text-expansion-query.html) query.
The mapping must include a field of type [`rank_features`](https://www.elastic.co/guide/en/elasticsearch/reference/current/rank-features.html) to work with our feature vectors of interest.
This field contains the token-weight pairs the ELSER model created based on the input text.

Let's create an index named `elser-movies` with the mappings we need.


In [ ]:
client.indices.create(
  index="elser-example-movies",
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 1,
          "default_pipeline": "elser-ingest-pipeline"
      }
  },
  mappings={
    "properties": {
      "plot": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "ml.tokens": {
        "type": "rank_features"
      },
    }
  }
)

## Insert Documents
Let's insert our example dataset of 12 movies.

In [ ]:
url = "https://raw.githubusercontent.com/leemthompo/notebook-tests/main/12-movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append({
        "_index": "elser-example-movies",
        "_source": doc,
    })

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `search-movies` index!")

Inspect a new document to confirm that it now has an `"ml": {"tokens":...}` field that contains a list of new, additional terms.
These terms are the **text expansion** of the field(s) you targeted for ELSER inference.
ELSER essentially creates a tree of expanded terms to improve the semantic searchability of your documents.
We'll be able to search these documents using a `text_expansion` query.

But first let's start with a simple keyword search, to see how ELSER delivers semantically relevant results out of the box.

# Querying Documents with ELSER

Let's test out semantic search using ELSER.

In [ ]:
response = client.search(
    index='elser-example-movies', 
    size=3,
    query={
        "text_expansion": {
            "ml.tokens": {
                "model_id":".elser_model_1",
                "model_text":"child toy"
            }
        }
    }
)

for hit in response['hits']['hits']:
    doc_id = hit['_id']
    score = hit['_score']
    title = hit['_source']['title']
    text = hit['_source']['text']
    print(f"Score: {score}\nTitle: {title}\nPlot: {text}\n")